In [2]:
import pandas as pd
import numpy as np

import random
import itertools
import pickle
from collections import OrderedDict, Counter

import matplotlib.pyplot as plt
import matplotlib.cm as cm

import seaborn as sns
sns.set()

# MODELS

from xgboost import XGBClassifier

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, StratifiedKFold, KFold
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, \
                                  VotingClassifier, \
                                  AdaBoostClassifier, BaggingRegressor


from sklearn.metrics import precision_score, recall_score, f1_score, fbeta_score, \
                            accuracy_score, roc_auc_score, make_scorer,\
                            confusion_matrix, precision_recall_curve, roc_curve

# MANAGING CLASS IMBALANCE

from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler # don't use this
from imblearn.pipeline import Pipeline, make_pipeline



# Notebook visuals

from mlxtend.plotting import plot_decision_regions
from mlxtend.classifier import StackingClassifier


plt.style.use('ggplot')
sns.set_style("whitegrid")
%matplotlib inline
%pylab inline
%config InlineBackend.figure_formats = ['retina']
%config InlineBackend.figure_format = 'svg' 

from ipywidgets import interactive, FloatSlider

# Make better use of Jupyter Notebook cell width

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


//anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# **Outline**

**Models**
- Logistic Regression (or SGDClassifier if you want to do elasticnet?)
- Naive Bayes
- KNN
- SVM
- Random forest and decision trees
- XGBoost

**Data setup**
- Feature engineering
- Train test split
- Stratification
- Class imbalance

**Cross validation**

**Ensembling** (once you have models in place)

**Model evaluation**
- ROC/AUC
- Precision, recall, f1 score
- Confusion matricies
- z score
- f beta
- MLE
- Predict probas are a crucial metric to include

#### Parameters you need to interrogate

- KNN # neighbors
- Logistic Regression C
- SVM C parameter
- Non-linear SVMs: kernel *and* gamma
- Trees: max_depth and n_estimators
- Ensembling: max, average, or weighted voting
- Oversampling: random, SMOTE, or ADASYN (you don't have to get to a perfect 50/50)
- **_threshold_**

#### Concepts to remember

- Always scale/standardize except for random forest
- Always regularize except for...
- Do not balance your validation data (this makes using normal CV models annoying)

#### this info needs to be incorporated but I don't know where yet
use special k-folds for imbalanced classes. stratified k-folds. shuffled = True
disadvantage to lassoCV, ridgeCV, you have less control 
do a for loop to make it happen
you can just do gridsearch cv and specify cross-validation <br> <br>

If you're not sure how to pick the best threshold for your model, you can spot check by printing out the precision_recall_curve in sklearn
precision_curve, recall_curve, threshold_curve = precision_recall_curve(y_test, lm.predict_proba(X_test)[:,1] )
df = pd.DataFrame(list(zip(precision_curve, recall_curve, threshold_curve)),
              columns=['precision','recall', 'threshhold'])
To hone in, try generating the plot to see where a good probable threshold lies


# Logistic Regression

Logistic regression uses the prediction function $f(\mathbf{x}) = \sigma(\mathbf{w}^\top \mathbf{x})$

In [3]:
# you can solve logistic regression with gradient descent (normal) and MLE


# Note that the regularization term C is always present.
# C = 0 means no regularization is happening. 


# solver options: liblinear, lbfgs
# saga is a good solver because it works for L1 and L2
from sklearn.linear_model import LogisticRegression


feature = train_df[['price_per_sqft']].values

lm1 = LogisticRegression(solver= 'liblinear', C=1000)

lm1.fit(X_train[['elevation']], y_train)
lm1.score(X_train[['elevation']], y_train)

prediction_hard = lm1.predict(feature)
prediction_soft = lm1.predict_proba(feature)[:, 1]


NameError: name 'train_df' is not defined

#### Use masks to illustrate predictions

In [ ]:

sf_mask = (train_df['location'] == 'SF').values
ny_mask = (train_df['location'] == 'NY').values

plt.plot(feature[sf_mask], prediction_hard[sf_mask], 'ro', label='Actually SF', alpha=0.1)
plt.plot(feature[ny_mask], prediction_hard[ny_mask], 'bo', label='Actually NY', alpha=0.1)
plt.xlabel('Price per square foot ($)')
plt.ylabel('Prediction (1=SF)');

#### Use predict_proba illustrate a different way

In [ ]:
plt.plot(feature[sf_mask], prediction_soft[sf_mask], 'ro', label='Actually SF', alpha = 0.1)
plt.plot(feature[~sf_mask], prediction_soft[~sf_mask], 'bo', label='Actually NY', alpha = 0.1)
plt.xlabel('Elevation (ft)')
plt.ylabel('Prob of being in SF');

You can get the coefficients and the intercept to back out exactly what your 
cutoffs are by reverse engineering the log odds. Look at [this link](https://stats.idre.ucla.edu/other/mult-pkg/faq/general/faq-how-do-i-interpret-odds-ratios-in-logistic-regression/)

In [ ]:
lm1.coef_, lm1.intercept_

#### Then you can do a confusion matrix. 

In [ ]:
confusion_matrix(y_train, prediction_hard)

## <font color='red'>Setting the threshold</font> 


# Naive Bayes

In [ ]:
nb = GaussianNB()
nb = MultinomialNB()
nb = BernoulliNB()

nb.fit(X_train, y_train)
nb.score(X_test, y_test)
nb.theta_ # mean of each feature by class
np.sqrt(nb.sigma_) # variance of each feature by class

# KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5) # weights is also a parameter
knn.fit(X_train, label_train)
print("The score for kNN is")
print("Training: {:6.2f}%".format(100*knn.score(X_train, label_train)))
print("Test set: {:6.2f}%".format(100*knn.score(X_test, label_test)))

# SVM
- The separating hyperplane is defined by the support vectors, which are the points near the plane that get moved around.
- C is a tuning parameter. Larger C tells the model it's OK to misclassify. This is what is called using a soft margin. 
    - Usually you find C with grid search and CV
- Gamma is a hyperparameter for non-linear kernels. It tells the model the kernel coefficient. Bigger gamma = more curviness. 

#### model

In [ ]:
# Kernels: linear, poly, rbf, probably more
# you can also change the degree with poly

svm_model = svm.SVC(kernel="linear")
svm_model.fit(x, y)

    
svm_model = svm.SVC(kernel=k_name, gamma=gamma, degree=degree, C=c, cache_size=1000, max_iter=1000)
n_sv = svm_model.support_vectors_.shape[0]


#### results

In [ ]:
print("support vectors: ", svm_model.support_vectors_)
print("coefficients: ", svm_model.dual_coef_)
print("intercept: ", svm_model.intercept_)

# Random Forest (and decision trees)

In [ ]:
decisiontree = DecisionTreeClassifier(max_depth=4)
randomforest = RandomForestClassifier(n_estimators=100)

# n estimators is the number of trees you can grow

# XGBoost
- Gradient boosted decision trees

# Data setup

#### Stratification

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2, stratify=y)

#### Train test split

In [ ]:
def get_user_split_data(df, test_size=.2, seed=42):

    rs = np.random.RandomState(seed)
    
    total_users = df['user_id'].unique() 
    test_users = rs.choice(total_users, 
                           size=int(total_users.shape[0] * test_size), 
                           replace=False)

    df_tr = df[~df['user_id'].isin(test_users)]
    df_te = df[df['user_id'].isin(test_users)] 

    y_tr, y_te = df_tr['in_cart'], df_te['in_cart']
    X_tr = df_tr.drop(['product_id','user_id','latest_cart','in_cart'],axis=1) 
    X_te = df_te.drop(['product_id','user_id','latest_cart','in_cart'],axis=1)

    return X_tr, X_te, y_tr, y_te

# Ensembling
- Two options for the final aggregate classifier: average voting or max voting

#### Bootstrapping
- samples are chosen *with replacement*

In [ ]:
# Run bagging classifier through cross validation
# ~45s to run
# instructor: pull up the bagging classifier docs if useful
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html

variance = []
bias = []
test_range = np.arange(1, 30, 1)

for i in test_range:
    cv_out = cross_validate(
        estimator=BaggingRegressor(
            DecisionTreeRegressor(random_state=123), n_estimators=i),
        X=X_train,
        y=y_train,
        cv=3,
        return_train_score=True,
        scoring={
            "variance": make_scorer(variance_metric),
            "bias": make_scorer(bias_metric)
        },
        n_jobs=-1,
    )
    variance.append(np.mean(cv_out['test_variance']))
    bias.append(np.mean(cv_out['test_bias']))

#### Bagging: Hard (max), soft (averaged), and weighted voting

In [ ]:
# create voting classifier

weights = ["pick out weights for the # of models you're testing"]
voting_classifer = VotingClassifier(estimators=model_list,
                                    voting='soft', #<-- sklearn calls average voting 
                                    n_jobs=-1      # soft voting and max voting
                                    weights = weights)     # not needed
voting_classifer.fit(X_train, y_train)             

In [ ]:
stacked = StackingClassifier(
    classifiers=model_vars, meta_classifier=BernoulliNB(), use_probas=False)
stacked.fit(X_train, y_train)

# Class imbalance

The following list is helpful when dealing with imbalanced datasets.

#### Do Nothing!
That's right, sometimes we'll get lucky and our classifier will deal effectively with the class imbalance. Go celebrate.

#### Balance the dataset in some way
- Would you run into _serious_ computational issues by doubling the amount of data you have? If so, use **Random Undersampling**
- If you have a lot variety in your dataset, you can try **Random Oversampling** as this method will generalize well from the minority observations you currently have.
- If Random Oversampling didn't work as well as you had hoped, try **generating synthetic data** with SMOTE or ADASYN. 


#### Switch to an Anomaly Detection Algorithm
If the above experiments don't yield the desired results, switch to an anomaly detection algorithm (not covered in this notebook).

#### Random oversampling, SMOTE, ADASYN

In [ ]:
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_sample(X,y)
Counter(y_resampled) # to see the new distribution of the two classes
clf_ros = SVC().fit(X_resampled, y_resampled)


X_smoted, y_smoted = SMOTE(random_state=42).fit_sample(X,y)
Counter(y_smoted) # to see the new distribution of the two classes
clf_smote = SVC().fit(X_smoted, y_smoted)

X_adasyn, y_adasyn = ADASYN(random_state=42).fit_sample(X,y)
Counter(y_adasyn)
clf_adasyn = SVC().fit(X_adasyn, y_adasyn)


# Cross validation
- Make sure to not balance your validation set! That means you can't use a lot of out-of-the-box CV methods

#### RandomizedSearchCV

In [ ]:
param_dist = dict(n_neighbors=k_range, weights=weight_options)
rand = RandomizedSearchCV(knn, param_dist, cv=10, scoring='accuracy', n_iter=10, random_state=42)
rand.fit(X, y)
print(rand.best_score_)
print(rand.best_params_)

#### cross_val_score

In [ ]:
# 3 lines of code
knn = KNeighborsClassifier(n_neighbors=5)
scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')
print(scores.mean())

# 2 lines of code, same concept as above
knn = KNeighborsClassifier(n_neighbors=20)
print(cross_val_score(knn, X, y, cv=10, scoring='accuracy').mean())

# same as above but also with a for loop to also search for # of k's
k_range = list(range(1, 31))
k_scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy') # CHANGE THIS, DUDE!
    k_scores.append(scores.mean())
print(k_scores)

# 10-fold cross-validation with logistic regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
print(cross_val_score(logreg, X, y, cv=10, scoring='accuracy').mean())

#### K Folds

In [ ]:
# K FOLDS!!

from sklearn.model_selection import KFold

X, y = cars.drop('price',axis=1), cars['price']

X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=10) #hold out 20% of the data for final testing

#this helps with the way kf will generate indices below
X, y = np.array(X), np.array(y)

#run the CV

kf = KFold(n_splits=5, shuffle=True, random_state = 42)
cv_lm_r2s, cv_lm_reg_r2s = [], [] #collect the validation results for both models

for train_ind, val_ind in kf.split(X,y):
    
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    #simple linear regression
    lm = LinearRegression()
    lm_reg = Ridge(alpha=1)

    lm.fit(X_train, y_train)
    cv_lm_r2s.append(lm.score(X_val, y_val))
    
    #ridge with feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    
    lm_reg.fit(X_train_scaled, y_train)
    cv_lm_reg_r2s.append(lm_reg.score(X_val_scaled, y_val))

print('Simple regression scores: ', cv_lm_r2s)
print('Ridge scores: ', cv_lm_reg_r2s, '\n')

print(f'Simple mean cv r^2: {np.mean(cv_lm_r2s):.3f} +- {np.std(cv_lm_r2s):.3f}')
print(f'Ridge mean cv r^2: {np.mean(cv_lm_reg_r2s):.3f} +- {np.std(cv_lm_reg_r2s):.3f}')

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state = 42)

print(np.mean(cross_val_score(lm, X, y, cv=kf, scoring='r2')))
print(np.mean(cross_val_score(lm_reg, X, y, cv=kf, scoring='r2')))

#### GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'alpha': np.linspace(0.0, 1.0, 100)} 
# keys have to be keyword arguments allowed in the model you're trying to fit
# e.g. double check docs that ridge uses 'alpha'
my_model = Ridge()
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
my_grid_search_ridge = GridSearchCV(my_model, param_grid, cv = 5, n_jobs = 1)
my_grid_search_ridge.fit(X_train_scaled, y_train)
my_grid_search_ridge.best_estimator_

# Feature engineering

Use consistent naming conventions for features of the same type. <br>
*Iterate*: Build features at the same level of aggregation at the same time, and track them in a dedicated dataframe. Merge back into the ML-formatted dataframe at the end of the process.

# Model evaluation

#### One-off scores
- Accuracy
- Balanced accuracy
- f1 and f-Beta
- log loss

#### MLE is another way of getting parameters

#### Predict probas

In [ ]:
probas = [c.predict_proba(X_train) for n,c in model_list]
probas += [voting_model.predict_proba(X_train)]

#### Confusion matricies

In [ ]:
def make_confusion_matrix(model, threshold=0.5):
    # Predict class 1 if probability of being in class 1 is greater than threshold
    # (model.predict(X_test) does this automatically with a threshold of 0.5)
    y_predict = (model.predict_proba(X_test)[:, 1] >= threshold)
    confusion_for_plot = confusion_matrix(y_test, y_predict)
    plt.figure(dpi=80)
    sns.heatmap(confusion_for_plot, cmap=plt.cm.Blues, annot=True, square=True, fmt='d',
           xticklabels=['legit', 'fraud'],
           yticklabels=['legit', 'fraud']);
    plt.xlabel('prediction')
    plt.ylabel('actual')

    
def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=18):
    """Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.
    
    Arguments
    ---------
    confusion_matrix: numpy.ndarray
        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. 
        Similarly constructed ndarrays can also be used.
    class_names: list
        An ordered list of class names, in the order they index the given confusion matrix.
    figsize: tuple
        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,
        the second determining the vertical size. Defaults to (10,7).
    fontsize: int
        Font size for axes labels. Defaults to 14.
        
    Returns
    -------
    matplotlib.figure.Figure
        The resulting confusion matrix figure
    """
    df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names, )
    fig = plt.figure(figsize=figsize)
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d")
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return fig

#### Precision and recall

In [ ]:
# generate plots
precision_curve, recall_curve, threshold_curve = precision_recall_curve(y_test, lm.predict_proba(X_test)[:,1] )

# precision recall curves separately
plt.figure(dpi=80)
plt.plot(threshold_curve, precision_curve[1:],label='precision')
plt.plot(threshold_curve, recall_curve[1:], label='recall')
plt.xlabel('Threshold (above this probability, label as fraud)');

# precision-recall curve
plt.figure(dpi=80)
plt.plot(recall_curve[1:], precision_curve[1:],label='precision')
plt.xlabel("Recall")
plt.ylabel("Precision");

# f1 score

#### ROC curve

# Asides

In [ ]:
# to make a new column in 1s and 0s for classificationb
data['Gender'] = (data['Gender'] != 'Male').astype(int)

iris_df = iris_df.query("species > 0")

x = iris_df[["petal length (cm)", "petal width (cm)"]].values
y = iris_df.species.values

In [ ]:
def scatter_plot_data(data):
    plt.figure(dpi=150)
    for ix, label in enumerate(['Male','Female']):
        new_data = data[data['Gender']==ix]
        plt.scatter(new_data['Height'], new_data['Weight'],c=plt.cm.jet(ix/0.5), alpha=0.4, label=label, s=5)
    plt.ylabel("Weight")
    plt.xlabel("Height")
    plt.legend(loc="upper left");

scatter_plot_data(data)

In [ ]:
# another way of plotting a simple scatter plot that I haven't seen
sns.lmplot('column1', 'column2', data=df, hue='label',
           palette='Set2', fit_reg=False, scatter_kws={'s': 20})
plt.gcf().set_size_inches(12,8);

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, lm.predict_proba(X_test)[:,1])

plt.plot(fpr, tpr,lw=2)
plt.plot([0,1],[0,1],c='violet',ls='--')
plt.xlim([-0.05,1.05])
plt.ylim([-0.05,1.05])


plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve for fraud problem');
print("ROC AUC score = ", roc_auc_score(y_test, lm.predict_proba(X_test)[:,1]))

In [ ]:
svm_model.support_vectors_ contains the support vectors. 
These are the same ones we found in our own solution above.
svm_model.dual_coef_ is what we called  𝐚  
and svm_model.intercept_ is what we called  𝑏 . 
These are different in scale, but otherwise quite 
similar to what we found above.

In [ ]:
Polynomial this is similar to using the PolynomialFeatures 
tool in preprocessing. It allows the model to make predictions 
based on higher order polynomial transformations of our input features.
RBF-Radial Basis Function this is similar to selecting 
examples as prototypes of a class. The radial basis function 
decreases as a test point gets farther away from this prototype in any direction 
(thus "radial").

In [4]:
# good for visualization 
sns.regplot('Date','Final_Time_Hund', data=horsey);

NameError: name 'horsey' is not defined